## Free coding assignment!

# Arrays

In [3]:
#include <iostream>
using std::cout, std::endl;

In [4]:
/*
    This method prints the address of the data
    i.e. where in memory does the "thing" reside
*/
template <class T>
void where(T const& thing) {
    printf("where: %p\n", &thing);
}

In [5]:
/*
  This method prints the bytes in RAM that a given variable has
  i.e. what are the actual 1s and 0s that occupy the space given to "thing"
  Prints the bytes in hexidecimal.
*/
template <class T>
void bytes(T const& thing) {
    unsigned char* addr = (unsigned char*)&thing;
    printf("bytes: 0x");
    for (int i = sizeof(T) - 1; i >= 0; i--) {
        printf("%02x", addr[i]);
    }
    printf("\n");
}

In [6]:
int foo = 7;
int bar = 4;

In [7]:
where(foo);
where(bar);

where: 0x7ffff87d602c
where: 0x7ffff87d6030


In [8]:
bytes(foo);

bytes: 0x00000007


In [16]:
int* fooptr = &foo;
where(foo);
bytes(fooptr);

where: 0x7ffff87d602c
bytes: 0x00007ffff87d602c


In [12]:
bytes(*fooptr);

bytes: 0x00000007


In [13]:
bytes(*(fooptr + 1))

bytes: 0x00000004


In [17]:
bytes(fooptr);
bytes(*fooptr);

fooptr++;  cout << "fooptr++" << endl;

bytes(fooptr);
bytes(*fooptr);
where(bar);

bytes: 0x00007ffff87d602c
bytes: 0x00000007
fooptr++
bytes: 0x00007ffff87d6030
bytes: 0x00000004
where: 0x7ffff87d6030


In [18]:
long long big_foo = 3;
long long big_bar = 5;

In [19]:
where(big_foo);
where(big_bar);

where: 0x7ffff87d6058
where: 0x7ffff87d6060


In [20]:
long long *big_foo_ptr = &big_foo;

bytes(big_foo_ptr);
bytes(*big_foo_ptr);

big_foo_ptr++;  cout << "big_foo_ptr++" << endl;

bytes(big_foo_ptr);
bytes(*big_foo_ptr);

bytes: 0x00007ffff87d6058
bytes: 0x0000000000000003
big_foo_ptr++
bytes: 0x00007ffff87d6060
bytes: 0x0000000000000005


When you "add one" to a pointer, you get a new memory address that is one type-width away from the first.

So adding one to an `int*` moves 4 bytes.

Adding one to a `long long*` moves 8 bytes.

In [22]:
int foo = 7;
int* fooptr = &foo;

where(*fooptr);
where(*(fooptr + 1));

where: 0x7ffff87d6080
where: 0x7ffff87d6084


In [23]:
where(fooptr[0]);
where(fooptr[1]);

where: 0x7ffff87d6080
where: 0x7ffff87d6084


In [24]:
where(fooptr[1]);

/* is equivalent to */

where(*(fooptr + 1));

where: 0x7ffff87d6084
where: 0x7ffff87d6084


if
```c++
int* foo;
```
then

```c++
foo[1]
```
is the same as saying:

> move 1 `int`s' worth of bytes from `foo` and get the value—i.e. `*(foo + 1)`

So, if you new you had a sequence of values all of the same type, all right next to each other in memory, you could easily get to all of them if you new the address of the first one.

<div style='font-size:200pt'>🤔</div>

## Array Allocation

In [25]:
int* quux = new int[10];

In [26]:
where(*quux)

where: 0x555558d14260


In [27]:
where(*(quux + 1))

where: 0x555558d14264


In [28]:
where(quux[0])

where: 0x555558d14260


In [29]:
where(quux[1])

where: 0x555558d14264


In [30]:
where(quux[2])

where: 0x555558d14268


- `new` array syntax
- `[]` on arrays does pointer math

In [31]:
bytes(quux[0])

bytes: 0x0deb9594


In [32]:
for (int i = 0; i < 10; i++) {
    bytes(quux[i]);
}

bytes: 0x0deb9594
bytes: 0x00005550
bytes: 0x00000000
bytes: 0x00000000
bytes: 0xffffffff
bytes: 0x00000000
bytes: 0x00000000
bytes: 0x00000000
bytes: 0x00000000
bytes: 0x00000000


- Calling `new T[n]` gives you space for `n` `T`s, but doesn't create any `T`s yet.
- There are bytes there, but they are just weeds growing in the empty lot you just purchased

In [37]:
for (int i = 0; i < 10; i++) {
    quux[i] = i;
}

In [40]:
for (int i = 0; i < 10; i++) {
    bytes(quux[i]);
}

bytes: 0x00000000
bytes: 0x00000001
bytes: 0x00000002
bytes: 0x00000003
bytes: 0x00000004
bytes: 0x00000005
bytes: 0x00000006
bytes: 0x00000007
bytes: 0x00000008
bytes: 0x00000009


In [41]:
for (int i = 10; i < 20; i++) {
    bytes(quux[i]);
}

bytes: 0x0000000a
bytes: 0x0000000b
bytes: 0x0000000c
bytes: 0x0000000d
bytes: 0x0000000e
bytes: 0x0000000f
bytes: 0x00000010
bytes: 0x00000011
bytes: 0x00000012
bytes: 0x00000013


Remember, the `[ ]` operator just does some pointer math and jumps to the specified slot in memory.

It doesn't know whether that slot in memory is available, has useful data, etc. 

If you used an array to store items under-the-hood, how would you implement:

- `push_back`
- `pop_back`
- `size`
- `push_front`
- `pop_front`


- preallocate an array that is "big enough"
- keep track of how many items you have
- add new items to the next available position in the array

**Issues**
- what happens with "big enough" isn't big enough anymore (i.e. you run out of room)?
  - do you stop adding items? (this is bad)
  - do you run off the end of the array? (this is REALLY bad)
- what happens if you loose track of how many items you have?


### Pro-tip 
> The goal is not to write code that you can get to work  
> The goal is to write code that you **can't get wrong**

## Vector

- Wrap an array
- Keep track of how big the array is and how many items we've added
- If we run out of room, we get a bigger array, copy the items over, and proceed
- always add the next item to the correct spot

## Big-O of Sequential Containers

| Operation     | SL List | DL List | Vector |
|---------------|------|--------|-------|
| `push_back`   |      |        |       |
| `pop_back`    |      |        |       |
| `push_front`  |      |        |       |
| `at`          |      |        |       |
| `insert(pos)` |      |        |       |

| Operation     | SL List | DL List | Vector |
|---------------|------|--------|-------|
| `push_back`   | O(n)     |   O(1)     | O(1)       |
| `pop_back`    | O(n)     |   O(1)     | O(1)      |
| `push_front`  | O(1)     |   O(1)     | O(n)      |
| `at`          | O(n)     |   O(n)     | O(1)      |
| `insert(pos)` | O(n)     |   O(n)     | O(n)      |

**Note**: Lists have $O(n)$ insert because it takes $O(n)$ time to find the position, but only $O(1)$ time to do the insert. If you assume the position is already known, it is accurate to say Lists have $O(1)$ insertion time. **State your assumptions.**

Vectors only take $O(1)$ time to find the position, but need $O(n)$ time to insert. 

What about space complexity?

### `space.cpp`

### `build_vector.cpp`

Write and discuss:

- Constructor and destructor
- `push_back` + `grow`
- `pop_back`
- `size`
- `operator []`

Note:
- `pop_back` just decrements size. Nothing is done to the memory.
- what is the significance of returning by value from `operator[]`?

## Key Ideas
- Arrays
  - contiguous memory allocation
- Vectors